<a href="https://colab.research.google.com/github/eminshall/Notebooks-Youtube-Channel/blob/main/Monte_Carlo_with_ARIMA_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime, date, timedelta
from statsmodels.tsa.arima.model import ARIMA
import datetime

symbol = 'CVX'
end = date.today()
start = end - datetime.timedelta(days = 7*365)


# gets the closing price fo Netflix for the past 7 years
my_stock = yf.Ticker(symbol)
my_stock = pd.DataFrame(my_stock.history(start = start, end = end)['Close'])
my_stock = my_stock.rename(str.lower, axis = 'columns')

# distribution for Monte Carlo simulation
my_stock_resid_distr_fitting = my_stock[-200:]
my_stock_train = my_stock[~my_stock.isin(my_stock_resid_distr_fitting).all(1)]
my_stock_test = my_stock_resid_distr_fitting[-100:]
my_stock_resid_distr_fitting = my_stock_resid_distr_fitting[~my_stock_resid_distr_fitting.isin(my_stock_test).all(1)]

# plots the my_stock weekly closing price over the past 5 years
plt.plot(my_stock.index, my_stock.close)
plt.title('Daily Closing Price: Past 7 Years')
plt.xlabel('Date')
plt.ylabel('Closing Share Price')
plt.show()

In [ ]:
# plots the ACF for the entire train period
pd.plotting.autocorrelation_plot(my_stock_train.close)
plt.show()

In [ ]:
# plots the ACF for the differenced data over the entire train period
pd.plotting.autocorrelation_plot(my_stock_train.close.diff().dropna())

In [ ]:
from statsmodels.tsa.stattools import adfuller

# performs ADF tests with no constant, a constant, and a constant plus linear trend on
for i in ['n', 'c', 'ct']:
    result = adfuller(my_stock_train.close, regression = i)
    print(f'ADF Statistic with %s for Closing {symbol} price: %f' % (i, result[0]))
    print('p-value: %f' % result[1])
    print('')

# performs ADF tests with no constant, a constant, and a constant plus linear trend on
# differenced 'symbol' closing share prices
for i in ['n', 'c', 'ct']:
    result = adfuller(my_stock_train.close.diff().dropna(), regression = i)
    print(f'ADF Statistic with %s for Differenced Closing {symbol} price: %f' % (i, result[0]))
    print('p-value: %f' % result[1])
    print('')

In [ ]:
history = np.log(my_stock_train.astype(float))
history = history.asfreq('d')

# fits the ARIMA model
my_stock_arima = ARIMA(np.log(my_stock_train.astype(float)), order = (1, 1, 1))
my_stock_arima_fit = my_stock_arima.fit()
print(my_stock_arima_fit.summary())

# plot residual errors of the training data
residuals = pd.DataFrame(my_stock_arima_fit.resid[2:])
residuals.plot()
plt.show()
residuals.plot(kind='kde')
plt.show()
print(residuals.describe())

In [ ]:
# creates a new dataframe that will be added to as the forecast rolls
history = np.log(my_stock_train.astype(float))
history.asfreq('d')
# creates an empty list that will hold predictions
predictions = []
# loops through the indexes of the set being forecased
for i in range(len(my_stock_resid_distr_fitting)):
    # estimates an ARIMA model of order (1,1,1)
    model = ARIMA(history, order = (1,1,1))
    # fits the model
    model_fit = model.fit()
    # forecasts the next period
    output = model_fit.forecast()
    # takes the predicted value and saves it in yhat
    yhat = np.e ** output.iloc[0]
    # appends yhat to the list of predictions
    predictions.append(yhat)
    # grabs the observation at the ith index
    obs = my_stock_resid_distr_fitting[i : i + 1]
    # appends the observation to the estimation data set
    history = pd.concat([history,np.log(obs.astype(float))])

# prints the MSE of the model for the rolling forecast period
error = mean_squared_error(my_stock_resid_distr_fitting, predictions)
print('Test MSE: %.3f' % error)

# converts the predictions list to a pandas dataframe with the same index as the actual values
# for plotting purposes
predictions = pd.DataFrame(predictions)
predictions.index = my_stock_resid_distr_fitting.index

# plots the predicted and actual stock prices
plt.plot(my_stock_resid_distr_fitting, color = 'blue')
plt.plot(predictions, color = 'red')
plt.xlabel('Date')
plt.ylabel('Dollars')
plt.title('Predicted vs. Actual Closing Weekly Stock Price')
plt.show()

In [ ]:
# plots the residuals
tune_residuals = my_stock_resid_distr_fitting.close - predictions[0]
tune_residuals.plot(kind = 'kde')

In [ ]:
from fitter import Fitter

f = Fitter(tune_residuals)
f.fit()
f.summary()

In [ ]:
warnings.filterwarnings('ignore')
# creates a function to do MC simulation with a Laplacian distribution
def laplace_monte_carlo(mean, residuals, n_sims):
    # gets the estimated beta or mean absolute distance from the mean
    beta = (sum(abs(residuals - np.mean(residuals)))
                           / len(residuals))
    # uses the numpy function to generate an array of simulated values
    est_range = np.random.laplace(mean, beta, n_sims)
    # converts the array to a list
    est_range = list(est_range)
    # returns the simulated values
    return(est_range)

def roll_forecast_nmc(train, test, std_dev, n_sims):
    # creates a new dataframe that will be added to as the forecast rolls
    history = np.log(train.astype(float))
    # creates an empty list that will hold predictions
    predictions = []
    # loops through the indexes of the set being forecased
    for i in range(len(test)):
        # estimates an ARIMA model of order (1,1,0)
        model = ARIMA(endog=history, order = (1,1,1))
        # fits the model
        model_fit = model.fit()
        # forecasts the next period
        output = model_fit.forecast()
        # takes the predicted value and saves it in yhat
        yhat = np.e ** output.iloc[0]
        # performs monte carlo simulation using the predicted price as the mean, user-specified
        # standard deviation, and number of simulations
        yhat_range = laplace_monte_carlo(yhat, std_dev, n_sims)
        # appends yhat_range to the list of predictions
        predictions.append([float(i) for i in yhat_range])
        # grabs the observation at the ith index
        obs = test[i : i + 1]
        # appends the observation to the estimation data set
        history = pd.concat([history, np.log(obs.astype(float))])

    # converts the predictions list to a pandas dataframe with the same index as the actual
    # values for plotting purposes
    predictions = pd.DataFrame(predictions)
    predictions.index = my_stock_resid_distr_fitting.index
    # converts all the estimated yhats in each column to one list per row
    predictions['predicted_range'] = predictions.values.tolist()
    # grabs only the column with all values in a list
    predictions = pd.DataFrame(predictions['predicted_range'])

    # returns predictions
    return(predictions)

# attaches the data withheld for investigating the forecast residuals back to the training
# data set to avoid a large error on the first forecast
my_stock_train = pd.concat([my_stock_train, my_stock_resid_distr_fitting])

tune_residuals = arima_model.resid.dropna()
# produces a rolling forecast with prediction intervals using 1000 MC sims
test_preds = roll_forecast_nmc(my_stock_train,
                               my_stock_test,
                               tune_residuals,
                              1000)


# creates an empty list
in_prediction_interval = []
# loops through the rows in the testing data set
for i in range(len(my_stock_test)):
    # appends true if the actual price is in the interval of predicted prices and false
    # otherwise
    in_prediction_interval.append(np.where(min(test_preds.predicted_range[i]) <=
                                           my_stock_test.close[i]
                                          <= max(test_preds.predicted_range[i]),
                                           True, False))
# prints the percentage of actual prices in the prediction intervals
print('Percentage of Actual Stock Prices in Predicted Stock Price Range: %f' %
      (100 * sum(in_prediction_interval) / len(in_prediction_interval)))

min_range = []
max_range = []

# loops through the rows in test_preds
for i in range(len(test_preds)):
    # appends to the list the min or max value as appropriate
    min_range.append(min(test_preds.predicted_range[i]))
    max_range.append(max(test_preds.predicted_range[i]))

# converts the lists to data frames and makes their indexes match up with the dates they're
# predicting
min_range = pd.DataFrame(min_range)
min_range.index = my_stock_test.index
max_range = pd.DataFrame(max_range)
max_range.index = my_stock_test.index

# plots the actual stock price with prediction intervals
plt.plot(my_stock_test[:], color = 'blue')
plt.plot(min_range[:], color = 'red')
plt.plot(max_range[:], color = 'red')
plt.xlabel('Date')
plt.ylabel('Dollars')
plt.title('Actual Closing Weekly Stock Price with Prediction Intervals')
plt.show()

#Consider Supporting Me

Making theses videos takes time and effort. Contributions from viewers like you help support my channel and avoid the need for promoting a Patreon, sponsors, or selling a course. Click the link and choose any amount you'd like. Thank you.

https://ko-fi.com/minshallsfi